In [2]:
import numpy as np
import networkx as nx
import cvxpy as cp
import pickle

# 读取保存的随机图
with open('random_graph.pkl', 'rb') as f:
    G, n, p = pickle.load(f)

# 获取节点权重
w = {str(node): G.nodes[node]['weight'] for node in G.nodes()}

print(f'图的点集为：{list(G.nodes)}')
print(f'图的边集为：{list(G.edges)}')
print(f'节点权重: {w}')

# 贪婪算法
def greedy_mis(graph, w):
    i_set = set()
    sorted_graph = sorted(graph, key=lambda node: w[str(node)], reverse=True)
    for v in sorted_graph:
        i_set.add(v)
        if len(graph[v]) != 0:
            for w in graph[v]:
                if w in sorted_graph:
                    sorted_graph.remove(w)
    return i_set

# Graph 转邻接矩阵
nodes = list(G.nodes())
num_nodes = len(nodes)
adj = np.zeros((num_nodes, num_nodes), dtype=int)
for edge in G.edges():
    node1, node2 = edge
    index1 = nodes.index(node1)
    index2 = nodes.index(node2)
    adj[index1, index2] = 1
    adj[index2, index1] = 1

# 邻接矩阵转邻接表
graph = dict([])
for i in range(num_nodes):
    graph[i] = []
    for j in range(num_nodes):
        if adj[i, j] == 1:
            graph[i].append(j)

# 使用贪婪算法求解最大独立集
greedy_is = greedy_mis(graph, w)
n = len(G.nodes)
binary_representation = [1 if i in greedy_is else 0 for i in range(n)]
binary_representation.reverse()
binary_string = ''.join(map(str, binary_representation))
A1 = binary_string
a1 = len(greedy_is)
Lower = 0
for i in range(len(greedy_is)):
    Lower += w[str(greedy_is.pop())]
print('贪心算法找到的下界：', A1)
print('贪心算法找到的下界点的个数 K1 为：', a1)
print('贪心算法找到的下界权重为：', Lower)

# Lovász 函数获取上界 a2
C = np.ones((len(G.nodes), len(G.nodes))) * max(w.values())
X = cp.Variable((n, n), PSD=True)
constraints = [cp.trace(X) == 1]
for i in range(len(adj)):
    for j in range(len(adj[0])):
        if adj[i][j] == 1:
            constraints.append(X[i][j] == 0)
prob = cp.Problem(cp.Maximize(cp.trace(C @ X)), constraints)
prob.solve()
print(X.value)
Upper = round(prob.value)
a2 = Upper // max(w.values())
print('半正定规划找到的上界点的 Kl 为：', a2)
print('半正定规划找到的上界权重为：', Upper)
print('-------------------------------------------------')

图的点集为：[0, 1, 2, 3, 4, 5, 6, 7]
图的边集为：[(0, 7), (1, 3), (1, 4), (2, 4), (3, 5), (5, 6), (5, 7)]
节点权重: {'0': 12, '1': 13, '2': 14, '3': 5, '4': 18, '5': 20, '6': 1, '7': 19}
贪心算法找到的下界： 00110001
贪心算法找到的下界点的个数 K1 为： 3
贪心算法找到的下界权重为： 50
[[ 1.40989692e-01  8.89585984e-02  9.37350954e-02  5.20313621e-02
   4.72548754e-02  6.74436617e-02  7.35463342e-02  7.68925861e-11]
 [ 8.89585984e-02  1.21525460e-01  1.21525934e-01  8.58211790e-11
  -9.33104053e-11  6.74437141e-02  5.40821035e-02  3.25672112e-02]
 [ 9.37350954e-02  1.21525934e-01  1.66951010e-01  4.54254371e-02
   8.40115266e-11  7.24384260e-02  9.45129392e-02  7.32162959e-02]
 [ 5.20313621e-02  8.58211790e-11  4.54254371e-02  1.28474498e-01
   8.30494414e-02 -8.11520120e-11  1.28474945e-01  7.64435181e-02]
 [ 4.72548754e-02 -9.33104053e-11  8.40115266e-11  8.30494414e-02
   8.30489812e-02 -4.99487516e-03  8.80441302e-02  3.57944359e-02]
 [ 6.74436617e-02  6.74437141e-02  7.24384260e-02 -8.11520120e-11
  -4.99487516e-03  6.74432546e-02  8.84